# Winery classification with the multivariate Gaussian

In this notebook, we return to winery classification, using the full set of 13 features.

## 1. Load in the data 

As usual, we start by loading in the Wine data set. Make sure the file `wine.data.txt` is in the same directory as this notebook.

Recall that there are 178 data points, each with 13 features and a label (1,2,3). As before, we will divide this into a training set of 130 points and a test set of 48 points.

In [15]:
# Standard includes
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
# Useful module for dealing with the Gaussian density
from scipy.stats import norm, multivariate_normal

In [16]:
# Load data set.
data = np.loadtxt('wine.data.txt', delimiter=',')
# Names of features
featurenames = ['Alcohol', 'Malic acid', 'Ash', 'Alcalinity of ash','Magnesium', 'Total phenols',
                'Flavanoids', 'Nonflavanoid phenols', 'Proanthocyanins', 'Color intensity', 'Hue',
                'OD280/OD315 of diluted wines', 'Proline']
# Split 178 instances into training set (trainx, trainy) of size 130 and test set (testx, testy) of size 48
np.random.seed(0)
perm = np.random.permutation(178)
trainx = data[perm[0:130],1:14]
trainy = data[perm[0:130],0]
testx = data[perm[130:178], 1:14]
testy = data[perm[130:178],0]

## 2. Fit a Gaussian generative model

We now define a function that fits a Gaussian generative model to the data.
For each class (`j=1,2,3`), we have:
* `pi[j]`: the class weight
* `mu[j,:]`: the mean, a 13-dimensional vector
* `sigma[j,:,:]`: the 13x13 covariance matrix

This means that `pi` is a 4x1 array (Python arrays are indexed starting at zero, and we aren't using `j=0`), `mu` is a 4x13 array and `sigma` is a 4x13x13 array.

In [17]:
def fit_generative_model(x,y):
    k = 3  # labels 1,2,...,k
    d = (x.shape)[1]  # number of features
    mu = np.zeros((k+1,d))
    sigma = np.zeros((k+1,d,d))
    pi = np.zeros(k+1)
    for label in range(1,k+1):
        indices = (y == label)
        mu[label] = np.mean(x[indices,:], axis=0)
        sigma[label] = np.cov(x[indices,:], rowvar=0, bias=1)
        pi[label] = float(sum(indices))/float(len(y))
    return mu, sigma, pi

In [18]:
# Fit a Gaussian generative model to the training data
mu, sigma, pi = fit_generative_model(trainx,trainy)

## 3. Use the model to make predictions on the test set

<font color="magenta">**For you to do**</font>: Define a general purpose testing routine that takes as input:
* the arrays `pi`, `mu`, `sigma` defining the generative model, as above
* the test set (points `tx` and labels `ty`)
* a list of features `features` (chosen from 0-12)

It should return the number of mistakes made by the generative model on the test data, *when restricted to the specified features*. For instance, using the just three features 2 (`'Ash'`), 4 (`'Magnesium'`) and 6 (`'Flavanoids'`) results in 7 mistakes (out of 48 test points), so

        `test_model(mu, sigma, pi, [2,4,6], testx, testy)`

should print 7/48.

**Hint:** The way you restrict attention to a subset of features is by choosing the corresponding coordinates of the full 13-dimensional mean and the appropriate submatrix of the full 13x13 covariance matrix.

In [19]:
from math import exp

# Now test the performance of a predictor based on a subset of features
def test_model(mu, sigma, pi, features, tx, ty):
    # print("Mu shape:", mu.shape)
    # print("Sigma shape:", sigma.shape)
    # print("Pi shape:", pi.shape)
    # print("X shape:", tx.shape)
    # print("Y shape:", ty.shape)
    # sigma = np.array([sigma[:, features, :][:, :, features]] * tx.shape[0])
    # mu = np.array([mu[:, features]] * tx.shape[0])
    # x = np.transpose(np.array([tx[:, features]] * mu.shape[1]), axes=[1, 0, 2])
    # print("Mu shape:", mu.shape)
    # print("Sigma shape:", sigma.shape)
    # print("Pi shape:", pi.shape)
    # print("X shape:", x.shape)
    # c = 1 / (((2 * pi) ** (len(features) / 2)) * abs(sigma) ** (1 / 2))
    # print("C shape", c.shape)
    # # p = c * exp(-(1 / 2) * np.transpose(x - mu) * np.negative(sigma) * (x - mu))
    # print(np.transpose(x - mu).shape)
    # print(np.dot(np.negative(sigma[1, 1, :, :]),(x - mu)[1, 1, :]).shape)
    # print(np.dot(np.transpose(np.negative(sigma)), (x - mu)).shape)
    # p = exp(-(1 / 2) * np.transpose(x - mu) * np.negative(sigma) * (x - mu))
    # print(p.shape)

    ### Your code goes here
    sigma = sigma[:, features, :][:, :, features]
    mu = mu[:, features]
    x = tx[:, features]

    print("Sigma shape:", sigma.shape)
    print("Mu shape:", mu.shape)
    print("X shape:", x.shape)
    print("P shape:", pi.shape)

    p = np.zeros((tx.shape[0], mu.shape[0], len(features)))
    for i in range(p.shape[0]):
        for j in range(p.shape[1]):
            if pi[j] != 0:
                c = 1 / (((2 * pi[j]) ** (len(features) / 2)) * np.linalg.det(sigma[j]) ** (1 / 2))
                print("C shape:", c.shape, ((2 * pi[j]) ** (len(features) / 2)).shape)
                exp_inside = -(1 / 2) * np.transpose(x[i] - mu[j]) * np.negative(sigma[j]) * (x[i] - mu[j])
                print(exp_inside.shape)
                pij = c * np.exp(exp_inside)
                print(pij.shape)
                p[i][j] = pij
    ###


### <font color="magenta">Fast exercises</font>

*Note down the answers to these questions. You will need to enter them as part of this week's assignment.*

Exercise 1. How many errors are made on the test set when using the single feature 'Ash'?

In [20]:
# test_model(mu, sigma, pi, [2], testx, testy)

Exercise 2. How many errors when using 'Alcohol' and 'Ash'?

In [21]:
test_model(mu, sigma, pi, [0,2], testx, testy)

Sigma shape: (4, 2, 2)
Mu shape: (4, 2)
X shape: (48, 2)
P shape: (4,)
C shape: () ()
(2, 2)
(2, 2)


ValueError: could not broadcast input array from shape (2,2) into shape (2)

Exercise 3. How many errors when using 'Alcohol', 'Ash', and 'Flavanoids'?

In [ ]:
test_model(mu, sigma, pi, [0,2,6], testx, testy)

Exercise 4. How many errors when using all 13 features?

In [ ]:
test_model(mu, sigma, pi, range(0,13), testx, testy)

Exercise 5. In lecture, we got somewhat different answers to these questions. Why do you think that might be?